In [5]:
from dell_runtime import DellRuntimeProvider
import os

In [6]:
provider = DellRuntimeProvider()

In [7]:
provider.remote(os.getenv("SERVER_URL"))

Opening webpage https://appsso.login.scfd.isus.emc.com/oauth/authorize?response_type=code&client_id=5c731039-4384-4ea1-b134-c9c9c8e25131&redirect_uri=http%3A%2F%2Fqdr-dev.oro-sandbox-small1.k8s.cec.lab.emc.com%2Fcallback&scope=openid+roles+user_attributes&state=R1qCNPZHMKBzxg2SK1xtag8V9GpKih



In [8]:
provider.runtime.programs()

{}

In [9]:
provider.runtime.pprint_programs()

In [10]:
RUNTIME_PROGRAM = """
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):

    user_messenger.publish({'results': 'intermittently'})

    circuits = transpile(
        circuits,
    )

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish({'results': 'finally'})
    user_messenger.publish(result.to_dict(), final=True)
    print("job complete successfully")
"""

In [11]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

In [12]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [13]:
proglist = provider.runtime.programs()
proglist

{'5a487b2140c8': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7fd3615fbf70>}

In [14]:
program_id

'5a487b2140c8'

In [15]:
provider.runtime.pprint_programs(refresh=True)

5a487b2140c8:
  Name: 5a487b2140c8
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [16]:
from qiskit import QuantumCircuit, execute, transpile
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [17]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

In [18]:
result = job.result(timeout=120)

In [19]:
job.status()

'Completed'